In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import os
import subprocess
from pathlib import Path
import tempfile
import time

import numpy as np
from tqdm.notebook import trange

%aimport minetest

repo_root = Path(globals()['_dh'][0]).parent.parent
temp_dir = tempfile.mkdtemp(prefix="minetest_")
game_dir = os.path.join(os.environ["CONDA_PREFIX"], "share/minetest/games/devtest")
# install devtest game if needed
if not os.path.exists(game_dir):
    os.makedirs(os.path.dirname(game_dir), exist_ok=True)
    os.symlink(os.path.realpath(os.path.join(repo_root, "games/devtest")), game_dir)

# use minetest from repo if needed
minetest_executable = os.path.join(os.environ["CONDA_PREFIX"], "bin/minetest")
if not os.path.exists(minetest_executable):
    minetest_executable = os.path.join(repo_root, "bin/minetest")
print(f"Using minetest executable: {minetest_executable}")

version_str = subprocess.check_output((minetest_executable, "--version"))
build_type_prefix = "BUILD_TYPE="
for line in version_str.decode().split("\n"):
    if line.startswith(build_type_prefix):
        build_type = line[len(build_type_prefix):]
        assert build_type == "Release", f"Benchmark should happen on a release build, got: {build_type}"

def benchmark(env, get_action):
    with env:
        env.reset()
        start = time.time()
        n_iters = 900
        for _ in trange(n_iters):
            action = get_action(env)
            observation, reward, done, truncated, info = env.step(action)
            if done:
                break
        stop = time.time()
        print("FPS:", n_iters / (stop - start))

def sample_action(env):
    return env.action_space.sample()

empty_action = {"KEYS": np.ndarray(0), "MOUSE": [0, 0]}


def get_empty_action(env):
    return empty_action

env = minetest.MinetestEnv(
    executable=minetest_executable,
    artifact_dir=os.path.join(temp_dir, "artifacts"),
    game_dir=game_dir,
    headless=True,
    log_to_stderr=True,
)

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/garymm
logging to /tmp/minetest_ls5omfpu/artifacts/log


Using minetest executable: /home/garymm/src/Astera-org/minetest/bin/minetest


## Sample action

In [3]:
benchmark(env, get_empty_action)

incrementing socket
Client started with pid 259725
capnp_client.init()...
Waiting for first obs...
Received first obs: (400, 600, 3)


  0%|          | 0/900 [00:00<?, ?it/s]

FPS: 59.81615590066839
